In [1]:
from bs4 import BeautifulSoup
import requests
%pip install selenium
%pip install webdriver_manager
%pip install firebase_admin

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

from firebase_admin import firestore
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("<your private key>")
firebase_admin.initialize_app(cred)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
df = pd.DataFrame(res)
df.to_csv('./Sw7up_Crawling_Information.csv')

                                               title      date  \
0                              2022년 1학기 학습동아리 선발 안내  22.03.01   
1  2022년 K-SW스퀘어 여름방학(8주) / 가을학기(16주) 프로그램 서류심사 합...  22.03.02   
2                       2022년 창업 액셀러레이션 참가팀 모집(~4/1)  22.03.02   
3   2022년 SW중심대학사업단 Git-Github 기초/고급 특강 신청 안내(~3/11)  22.03.04   
4                          2022년 1학기 학부생 TA 추가 모집 안내  22.03.04   

                                                 url  
0  https://sw7up.cbnu.ac.kr/community/notice/621d...  
1  https://sw7up.cbnu.ac.kr/community/notice/621e...  
2  https://sw7up.cbnu.ac.kr/community/notice/621e...  
3  https://sw7up.cbnu.ac.kr/community/notice/6221...  
4  https://sw7up.cbnu.ac.kr/community/notice/6221...  


In [16]:
def saveData(data, length):
    df = pd.DataFrame(data)
    df.to_csv('./Sw7up_Crawling_Information.csv', mode='a')
    
    db = firestore.client()
    doc_ref = db.collection(u'sw7up').document(u'%04d' % length)
    doc_ref.set({'title': data['title'], 
                'date': data['date'],
                'url': data['url']})

def notification(data):
    print("노티를 보냈습니다")
    print(data)

In [46]:
def Crawling():
    browser = webdriver.Chrome(ChromeDriverManager().install())
    docs = pd.read_csv('./Sw7up_Crawling_Information.csv')
    length = len(docs['title'])
    
    isNew = False
    res = {'title': [], 'date': [], 'url': []}
    
    page = f"https://sw7up.cbnu.ac.kr/community/notice?page=1&limit=10&sort=-createdAt"
    html = requests.get(page).text
    bsObject = BeautifulSoup(html, "html.parser")

    for link in bsObject.find_all('span', 'mb-0'):
        title = str(link).split('>')[1].split('<')[0]
        isNew = True
        res['title'].append(title)
            
            
    for link in bsObject.find_all('small')[2::3]:
        res['date'].append(str(link).split('>')[1].split('.<')[0])
        
    time.sleep(5)
    browser.get(page)
    for i in range(2):
        elem = browser.find_elements(By.CLASS_NAME, 'card')
        time.sleep(3)
        elem[i].click()
        print(i)
        time.sleep(3)
            
        res['url'].append(browser.current_url)
        browser.back()
        time.sleep(3)
    
    res['title'] = res['title'][:2]
    res['date'] = res['date'][:2]
    df = pd.DataFrame(res)
    for i in reversed(range(len(res['title']))):
        title = res['title'][i]
        if title in docs['title'].to_list():
            continue
        
        print(title)
        # notification(res[i])
        pd.DataFrame(df.iloc[i]).transpose().to_csv('./Sw7up_Crawling_Information.csv', mode='a', header=False)
        #aveData(res[i], length)
        length += 1

In [51]:
import pandas as pd
Crawling()

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_24724\710670988.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


0
1
2022년 클라우드 특강 신청 안내(~12/15)


In [ ]:
while(1):
    Crawling()
    time.sleep(3600)